In [3]:
import numpy as np
import warnings
import pandas as pd
import os

In [4]:
from load_contracts import read_contract, preprocess_text
from load_info import get_security_names, load_info_xlsx
from manage_triggers import get_proccessed_triggers

In [5]:
filename = "contracts/135_ActelisNetworks_COI_01072005.pdf"

In [6]:
BEGINING_OF_IV_INTRO_TEXT = get_proccessed_triggers("Begining_IV")
END_OF_IV_INTRO_TEXT = get_proccessed_triggers("End_IV_intro")

### Utils

In [115]:
def find_loc(text, triggers, allow_contains=True):
    for trigger in triggers:
        curIndex = find_target(text, trigger, allow_contains=allow_contains)
        if curIndex != []:
            if len(curIndex) > 1:
                pass
#                 print("target occurs at multiple indicies")
            return curIndex
#     print("No trigger found in text")
    return None

In [92]:
def get_filepath(target_filename):
    dropbox_path = "/Users/DanielLongo/Dropbox/VC RA Avinika Narayan/Contracts project/coi/Done OCR'd/"
    for root, dirs, files in os.walk(dropbox_path):
        for dir in dirs:
            for filename in os.listdir(dropbox_path + dir):
                if filename == target_filename:
                    return dropbox_path + dir + "/" + filename
#     print(target_filename, "not found")

In [6]:
get_filepath("17445_955Dreams_COI_10112013.pdf")

"/Users/DanielLongo/Dropbox/VC RA Avinika Narayan/Contracts project/coi/Done OCR'd/955 Dreams/17445_955Dreams_COI_10112013.pdf"

In [7]:
def remove_periods(text): #doesn't remove decimal places
    out = []
    for word in text:
        if is_num(word):
            out += [word]
            continue
        out += word.split('.')
    return out

In [ ]:
# def get_closest 

In [8]:
def is_num(x):
    x = x.strip('$')
    try:
        float(x)
    except ValueError:
        return False
    return True

In [9]:
def revert_secuirty_names(df):
    df.loc[df["Security Name"] == "common stock", "Security Name"] = "common"
    return df

In [155]:
def get_closest_string(values, target, less, any=False):
    if not any:
        if less:
            values = [x for x in values if x < target]
        else:
            values = [x for x in values if x > target]
    try:
        val = min(values, key=lambda x: abs(target - x))
    except ValueError:
        return -1
    return val

In [75]:
def find_target(text, target, allow_contains=True):
    target_loc = 0
    indicies = []
    start_index = 0
    for i in range(len(text)):
        word = text[i]
#         print("target_loc", target_loc)
        if word == target[target_loc] or (allow_contains and word in target[target_loc]):
            if target_loc == 0: #marks start of sequence
                start_index = i
            target_loc += 1
            if target_loc == len(target):
                indicies += [start_index]
                target_loc = 0
        elif word == target[0] or (allow_contains and word in target[0]):
            target_loc = 1
        else:
            if target_loc > 0:
                #reset just incase ababcd 
                i -= (target_loc - 1) 
            target_loc = 0
    return indicies

### Search Functions

In [76]:
def get_closest_num(values, target, less, min=-1, max=1e10000, any=False):
    try:
        index = int(target)
    except ValueError:
        indicies = find_target(values, target.split(" "))
        if len(indicies) > 1:
            warnings.warn("Indicies length greath than one", RuntimeWarning)
            for i in range(len(indicies)):
                if less:
                    cur_target = indicies[i]
                else:
                    cur_target = indicies[len(indicies) - 1 -i] #starts at end of doc
                cur_val = get_closest_num(values, cur_target, less, min=min, max=max, any=any)
                if (cur_val != None):
                    return cur_val
            warnings.warn("No number found", RuntimeWarning)
            print("No num found target:", target, "indicies:", indicies)
            return None
        assert(len(indicies) > 1), "Invalid target"
        index = indicies[-1]
    if any: #doesn't get closest
        iters = range(0, len(values))
    elif less:
        iters = range(index -1, -1, -1) #0 is last
    else:#less = false
        iters = range(index, len(values))
    for i in iters:
        if (is_num(values[i])):
            return int(values[i])
    print("No num", target, "index", index)
    return None
    

In [77]:
def get_names(text):
    unsplit_names = get_security_names()
    names = [name.split(" ") for name in unsplit_names]
    indicies = []
    types = []
    for name in names:
        if (len(name) == 1):
            name += ["stock"]
        cur_indicies = find_target(text, name)
        if len(cur_indicies) > 0:
            types += [" ".join(name)]
            indicies += [cur_indicies]
    #remove extra "prefered stock"
    has_extra_prefered = True
    if "prefer stock" in types:
        for type in types:
            if type[-6:] == 'prefer':
                has_extra_prefered = True
                break
        if (has_extra_prefered):
            indicies.pop(types.index('prefer stock'))
            types.remove('prefer stock')
    return indicies, types

In [78]:
def get_num_of_shares(text, names):
    out = []
    for name in names:
        out += [get_closest_num(text, name, less=True)]
    return out
        

In [79]:
def get_types(names):
    types = []
    for name in names:
        if "common" in name:
            types += ["common"]
        elif "prefer" in name:
            types += ["prefered"]
        else:
            print("name doesn't contain common or prefer:", "".join(name))
            types += ["No type found"]
    return types

In [80]:
def generate_stats(IV_intro_text):
    _, names = get_names(IV_intro_text)
    types = get_types(names)
    nums = get_num_of_shares(IV_intro_text, names)
    d = {"Security Name":names, "Security Type":types, "Number":nums}
    df = pd.DataFrame(d)
    df = revert_secuirty_names(df)
    return df

In [81]:
unsplit_names = get_security_names()

In [82]:
all_names = [name.split(" ") for name in unsplit_names]

In [83]:
indicies = []
for name in all_names:
    indicies += [find_target(text, name)]


NameError: name 'text' is not defined

In [25]:
index_of_article_IV = find_target(text, ["articl", "fourth"])

In [26]:
text[index_of_article_IV[0]: index_of_article_IV[0] + 2]

IndexError: list index out of range

### Search Through IV Intro

In [27]:
end_of_IV_intro = find_target(text, END_OF_IV_INTRO_TEXT)

In [28]:
assert(len(end_of_IV_intro) == 1)

AssertionError: 

In [29]:
end_of_IV_intro = end_of_IV_intro[0]

IndexError: list index out of range

In [22]:
articl_forth_occur = find_target(text, ["articl", "fourth"])

In [23]:
begining_of_IV = get_closest_string(values = articl_forth_occur, target=end_of_IV_intro, less=True)

In [24]:
print("IV Intro starts", begining_of_IV, "IV Intro ends", end_of_IV_intro)

IV Intro starts 203 IV Intro ends 363


In [25]:
" ".join(text[begining_of_IV:end_of_IV_intro])

'articl fourth compani is author to issu two class of share design respect common stock and prefer stock the compani is author to issu 43000000 share of common stock with a par valu of $0.01 per share the compani is author to issu 26007500 share of prefer stock 800000 of which are design seri a prefer stock with a par valu of $0.01 per share seri a prefer 700000 of which are design seri b prefer stock with a par valu of $0.01 per share seri b prefer 1507500 of which are design seri c prefer stock with a par valu of $0.01 per share seri c prefer and 23000000 of which are design seri d prefer stock with a par valu of $0.01 per share seri d prefer for the purpos of thi amend and restat certif of incorpor the term prefer stock shall mean seri a prefer seri b prefer seri c prefer and seri d prefer collectively.th'

In [26]:
IV_intro_text = text[begining_of_IV:end_of_IV_intro]

In [27]:
indicies, names = get_names(IV_intro_text)

In [28]:
types = get_types(names)

In [29]:
get_closest_num(IV_intro_text, names[0], False)

26007500

In [30]:
get_closest_num(IV_intro_text, names[0], False)

26007500

In [31]:
nums = get_num_of_shares(IV_intro_text, names)

No num 13 index 13


In [32]:
print(names)
print(types)
print(nums)

['common stock', 'seri a prefer', 'seri b prefer', 'seri c prefer', 'seri d prefer']
['common', 'prefered', 'prefered', 'prefered', 'prefered']
[43000000, 800000, 700000, 1507500, 23000000]


In [33]:
generate_stats(IV_intro_text)

No num 13 index 13


,Security Name,Security Type,Number
0,common,common,43000000
1,seri a prefer,prefered,800000
2,seri b prefer,prefered,700000
3,seri c prefer,prefered,1507500
4,seri d prefer,prefered,23000000


### Evalutate Search Methods

In [171]:
def get_IV_intro_text(filename, begining_intro_triggers, end_intro_triggers):
    text = read_contract(filename)
    text = text.split(" ")
#     print("text", text)
#     text = text.split(" ")
#     text = remove_periods(text)
        
    if filename in Failures:
        return "Failure expected"
    try:
        begining_of_IV_intro = find_loc(text, begining_intro_triggers, allow_contains=False)
        end_of_IV_intro = find_loc(text, end_intro_triggers)[0]
#         print("end IV sample", " ".join(text[end_of_IV_intro: end_of_IV_intro + 50])
#         end_of_IV_intro = end_of_IV_intro[0]
#         end_of_IV_intro = 
    except TypeError:
        print("failed", filename)
        print(text)
        return None
    assert(begining_of_IV_intro != None), "Invalid info file " + " ".join(text) + filename
        
#     assert(end_of_IV_intro != None), "END Sequence not found"
#     articl_forth_occur = find_target(text, ["articl", "fourth"])
#     assert(articl_forth_occur != None), "Article fourth Sequence not found"
    begining_of_IV = get_closest_string(values = begining_of_IV_intro, target=end_of_IV_intro, less=True)
    if begining_of_IV == -1:
        print("no begining found")
        begining_of_IV = end_of_IV_intro - 100
    print("begining", begining_of_IV, "end", end_of_IV_intro)
    IV_intro_text = text[begining_of_IV:end_of_IV_intro]
    return IV_intro_text

In [176]:
def test_methods(filename, n=-1, start_index=0):
    iters = 0
    y = load_info_xlsx("rs1 database AN.xlsx", np_array=False)
    prev_filename = ""
    s, f = 0, 0
    for filename in y["File Name"]:
        if (start_index > 0):
            start_index -= 1
            continue
        if filename == prev_filename:
            continue
        prev_filename = filename
        iters += 1
        if iters == n:
            break
#         print(filename)
        filepath = get_filepath(filename)
        if filepath == None:
            continue
            
        #Test load intro text   
        Intro_IV_text = get_IV_intro_text(filepath, BEGINING_OF_IV_INTRO_TEXT, END_OF_IV_INTRO_TEXT)
        if (Intro_IV_text != None):
            s += 1
        else:
            return
            f += 1
        print(" ".join(Intro_IV_text))
        #Test load stats
#         stats = generate_stats(Intro_IV_text)
#         print(stats)
            
    print("SUCCESS", s)
    print("FAILURE", f)
        
#     text = get_IV_intro_text(filename)
#     stats = generate_stats(text)
    
    

In [177]:
test_methods(filename, n=-1, start_index=0)

no begining found
begining 489 end 589
gener corpor law articl iv author shares th total number of share of all class o f stock which the corpor shall have author to issu is a twelv million 12000000 share of common stock $0.0001 par valu per share common stock and b two million eight hundr thousand 2800000 share o f prefer stock $0.0001 par valu per share prefer stock a of the effect date of thi restat certif of incorpor thi restat certif all two million eight hundr thousand 2800000 share o f the author prefer stock of the corpor are herebi design seri seed prefer stock
no begining found
begining 519 end 619
author to issu is a 15000000 share of common stock $0.0001 par valu per share common stock and b 7021906 share of prefer stock $0.0001 par valu per share prefer stock a of the effect date of thi third restat certif of incorpor thi restat certif 2717681 share of the author prefer stock of the corpor are herebi design seri seed prefer stock 1 414225 share of the author prefer stock o

begining 621 end 3072
iv shall not advers affect ani contract or other right to indemnif of ani agent of the corpor that exist at or prior to the time of such amend repeal or modification articl v1 author of share thi corpor is author to issu two 2 class of share design common stock and prefer stock respect both of which shall have no par valu the total number of share of common stock author to be issu is170000000 share the total number of share of prefer stock author to be issu is 116655843 share 35892856 of which are design seri a prefer stock 43262987 of which are design seri b prefer stock and 37500000 of which are design seri c prefer stock 1 varticl vith follow is a statement of the right prefer privileg and restrict grant to and impos on each class of capit stock of the corpor unless otherwis indic refer to section or subsect in thi articl refer to section and subsect of thi articl vi a common stock1 gener the vote dividend and liquid right of the holder o f common stock are su

begining 14648 end 15687
fourth the seri a prefer stock until all share of seri a prefer stock subject to redempt have been redeem and shall redeem the remain share to have been redeem a soon a practic after the corpor ha fund legal avail therefor thi corpor shall not redeem ani other share of capit stock of the corpor other than secur repurchas from former employe offic director consult or other person who perform servic for the corpor or ani subsidiari in connect with the cessat of such employ or servic at the lower of fair market vaiue or the origin purchas price thereof prior to the redempt of all share of prefer stock pursuant to thi subsect b 6 v except for exclud share a defin below unless otherwis determin by the prior vote or written consent of the holder o f at least a major of the prefer slock on an a convert to common stock basi the redempt price for seri d prefer stock shall mean the greater of a the seri d origin issu price plu all declar but unpaid dividend on a share of

KeyboardInterrupt: 

In [84]:
BEGINING_OF_IV_INTRO_TEXT = get_proccessed_triggers("Begining_IV")
END_OF_IV_INTRO_TEXT = get_proccessed_triggers("End_IV_intro")
print(BEGINING_OF_IV_INTRO_TEXT)
print(END_OF_IV_INTRO_TEXT)

[['fourth'], ['iv'], ['corpor', 'is', 'author', 'to', 'issu'], ['the', 'total', 'number', 'of', 'share', 'of', 'all', 'class', 'of', 'stock']]
[['the', 'follow', 'is', 'a', 'statement', 'of', 'the', 'design', 'and', 'the', 'right', 'power', 'and', 'prefer'], ['follow', 'is', 'a', 'statement', 'of', 'the', 'design', 'and', 'the', 'right', 'power', 'and', 'prefer'], ['rel', 'right', 'prefer', 'privileg', 'and', 'restrict', 'grant'], ['section', '1', 'dividen'], ['the', 'undersign', 'incorpor'], ['power', 'prefer', 'right', 'restrict', 'and', 'other', 'matter'], ['dividend'], ['amend', 'to', 'restat', 'certif', 'of', 'incorpor', 'ha', 'been', 'duli', 'adopt'], ['fith'], ['the', 'name', 'and', 'mail', 'address', 'of', 'the', 'sole']]


In [1]:
Failures = [
    "/Users/DanielLongo/Dropbox/VC RA Avinika Narayan/Contracts project/coi/Done OCR'd/ActiveSemi International/148_ActiveSemi_COI_11082010.pdf",
]
